In [1]:
import pandas as pd
import time

In [5]:
with open('onegin.txt') as f:
  phrases = [s.replace('\t\t','') for s in f.read().split('\n') if s.startswith('\t\t')]

In [6]:
phrases

['He мысля гордый свет забавить,',
 'Вниманье дружбы возлюбя,',
 'Хотел бы я тебе представить',
 'Залог достойнее тебя,',
 'Достойнее души прекрасной,',
 'Святой исполненной мечты,',
 'Поэзии живой и ясной,',
 'Высоких дум и простоты;',
 'Но так и быть – рукой пристрастной',
 'Прими собранье пёстрых глав,',
 'Полусмешных, полупечальных,',
 'Простонародных, идеальных,',
 'Небрежный плод моих забав,',
 'Бессонниц, лёгких вдохновений,',
 'Незрелых и увядших лет,',
 'Ума холодных наблюдений',
 'И сердца горестных замет.',
 '«Мой дядя самых честных правил,',
 'Когда не в шутку занемог,',
 'Он уважать себя заставил',
 'И лучше выдумать не мог.',
 'Его пример другим наука;',
 'Но, боже мой, какая скука',
 'С больным сидеть и день и ночь,',
 'Не отходя ни шагу прочь!',
 'Какое низкое коварство',
 'Полуживого забавлять,',
 'Ему подушки поправлять,',
 'Печально подносить лекарство,',
 'Вздыхать и думать про себя:',
 'Когда же чёрт возьмёт тебя!»',
 'Так думал молодой повеса,',
 'Летя в пыли на п

In [7]:
phrases[:10]

['He мысля гордый свет забавить,',
 'Вниманье дружбы возлюбя,',
 'Хотел бы я тебе представить',
 'Залог достойнее тебя,',
 'Достойнее души прекрасной,',
 'Святой исполненной мечты,',
 'Поэзии живой и ясной,',
 'Высоких дум и простоты;',
 'Но так и быть – рукой пристрастной',
 'Прими собранье пёстрых глав,']

In [8]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [9]:
CHARS = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')

In [10]:
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]

In [11]:
INDEX_TO_CHAR

['none',
 ' ',
 'м',
 'ж',
 'в',
 'а',
 'о',
 'и',
 'ё',
 'х',
 'с',
 'ю',
 'ц',
 'т',
 'г',
 'з',
 'ъ',
 'й',
 'п',
 'ы',
 'щ',
 'б',
 'л',
 'д',
 'н',
 'ш',
 'р',
 'э',
 'у',
 'ф',
 'к',
 'ч',
 'я',
 'ь',
 'е']

In [12]:
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [13]:
CHAR_TO_INDEX

{'none': 0,
 ' ': 1,
 'м': 2,
 'ж': 3,
 'в': 4,
 'а': 5,
 'о': 6,
 'и': 7,
 'ё': 8,
 'х': 9,
 'с': 10,
 'ю': 11,
 'ц': 12,
 'т': 13,
 'г': 14,
 'з': 15,
 'ъ': 16,
 'й': 17,
 'п': 18,
 'ы': 19,
 'щ': 20,
 'б': 21,
 'л': 22,
 'д': 23,
 'н': 24,
 'ш': 25,
 'р': 26,
 'э': 27,
 'у': 28,
 'ф': 29,
 'к': 30,
 'ч': 31,
 'я': 32,
 'ь': 33,
 'е': 34}

In [14]:
import torch

In [28]:
MAX_LEN = 30

In [29]:
X = torch.zeros((len(text), MAX_LEN), dtype=int)

In [30]:
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [31]:
X[0:1]

tensor([[ 0,  0,  1,  2, 19, 10, 22, 32,  1, 14,  6, 26, 23, 19, 17,  1, 10,  4,
         34, 13,  1, 15,  5, 21,  5,  4,  7, 13, 33,  0]])

In [32]:
class Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.RNN(35, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [80]:
class NetworkGRU(torch.nn.Module):

    def __init__(self):
        super(NetworkGRU, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.GRU(35, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [81]:
class NetworkLSTM(torch.nn.Module):

    def __init__(self):
        super(NetworkLSTM, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.lstm = torch.nn.LSTM(35, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.lstm(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.lstm(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [33]:
model = Network()

In [34]:
model.forward(X[0:1])[0].shape

torch.Size([1, 30, 35])

In [55]:
def generate_sentence(mdl):
    sentence = ['я', ' ', 'в', 'а', 'с']
    #sentence = ['а', 'х', ' ', 'н', 'о', 'ж', 'к', 'и']
    state = None
    for i in range(MAX_LEN):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long)
        if i == 0:
            result, state = mdl.forward(X)
        else:
            result, state = mdl.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(INDEX_TO_CHAR[index_of_prediction])
    print(''.join(sentence))

In [43]:
generate_sentence(model)

я вас полтпосаоо т   носссейтттр nonenone


In [56]:
def learning_loop(mdl, epochs=2000, oprimizer):
    criterion = torch.nn.CrossEntropyLoss()
    for ep in range(epochs):
        start = time.time()
        train_loss = 0.
        train_passed = 0

        for i in range(int(len(X) / 100)):
            batch = X[i * 100:(i + 1) * 100]
            X_batch = batch[:, :-1]
            Y_batch = batch[:, 1:].flatten()

            optimizer.zero_grad()
            answers, _ = mdl.forward(X_batch)
            answers = answers.view(-1, len(INDEX_TO_CHAR))
            loss = criterion(answers, Y_batch)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()
            train_passed += 1

        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        generate_sentence(mdl)

In [57]:
learning_loop(model, epochs=2000, optimizer=torch.optim.SGD(model.parameters(), lr=.5))

Epoch 0. Time: 0.273, Train loss: 1.094
я вас псс ноевеев ищтnoneт ееnoneомг noneвnoneл
Epoch 1. Time: 0.238, Train loss: 1.115
я вас псс поевлелдсечиnoneь шй  меонне
Epoch 2. Time: 0.228, Train loss: 1.134
я вас псс поевлелдсечиыь шйnone ууоозт
Epoch 3. Time: 0.248, Train loss: 1.192
я васмпссnoneеатьаотеnonenone еnonenonenonenoneгnonenonenonenoneо nonenone
Epoch 4. Time: 0.243, Train loss: 1.215
я вас псс пртаоееер  тnoneиинртмн ыиаа
Epoch 5. Time: 0.248, Train loss: 1.313
я вас пыс прлалоееди нnoneнтnoneе аи  длн
Epoch 6. Time: 0.215, Train loss: 1.205
я вас псс поеапелдлерьnoneиnoneнnonenone noneяnonenoneиnone
Epoch 7. Time: 0.249, Train loss: 1.156
я вас пссnoneпоевnoneикма лаолпудбееш ут 
Epoch 8. Time: 0.249, Train loss: 1.140
я вас пысnoneпомлееры ннnone баа гыллзлnoneа
Epoch 9. Time: 0.242, Train loss: 1.236
я вас сыс побттолрои иейnoneс  none аоnoneлв
Epoch 10. Time: 0.214, Train loss: 1.226
я вас сыс гухатллнnoneеиаоnoneн    аснннл
Epoch 11. Time: 0.247, Train loss: 1.199
я 

In [83]:
model = NetworkGRU()
learning_loop(model, epochs=2000, optimizer=torch.optim.Adam(model.parameters(), lr=.5))

Epoch 0. Time: 0.672, Train loss: 20.266
я васnonenonenonenoneоnonenonenonenoneоnonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 1. Time: 0.663, Train loss: 17.139
я васцnoneшэкnonenonenoneууаnonenoneуооnonenoneурррnoneуnonenonenonenoneне
Epoch 2. Time: 0.643, Train loss: 12.435
я васчnoneизаа   м ииnonenoneлллnonenonenoneое nonenone псnone
Epoch 3. Time: 0.633, Train loss: 7.893
я васеnoneеллиnoneльь noneьnonenoneнnone nonenoneаnoneнnonenonenone  nonenone
Epoch 4. Time: 0.659, Train loss: 7.127
я васнбынллзnonenoneииnonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 5. Time: 0.667, Train loss: 6.944
я васнмрриаиоогсхддnonenonenoneвnonenonenonenonenonenonenonenonenonenonenonenone
Epoch 6. Time: 0.652, Train loss: 7.905
я васзбирпирлоорnonenoneввяnonenonenonenoneлnonenonenonenonenonenonenonenonenone
Epoch 7. Time: 0.686, Train loss: 6.708
я васанылллллииииилллллииииилллллии
Epoch 8. Time: 0.703, Train loss: 6.430
я ва

In [84]:
model = NetworkLSTM()
learning_loop(model, epochs=2000, optimizer=torch.optim.SGD(model.parameters(), lr=.5))

Epoch 0. Time: 1.420, Train loss: 3.099
я васnonenone none nonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 1. Time: 1.323, Train loss: 2.772
я вас                              
Epoch 2. Time: 1.365, Train loss: 2.679
я вас      ооооо      ооооо      оо
Epoch 3. Time: 1.312, Train loss: 2.626
я вас оо  о  оо оо  о  оо оо  о  оо
Epoch 4. Time: 1.375, Train loss: 2.586
я вас оо  о  оо оо  о  оо оо  о  оо
Epoch 5. Time: 1.292, Train loss: 2.549
я вас оо  о  оо ооо о   о ооооо    
Epoch 6. Time: 1.309, Train loss: 2.512
я вас оо  о  сосооо     сооооо     
Epoch 7. Time: 1.341, Train loss: 2.476
я вас со  ст ссоооот     ооооос    
Epoch 8. Time: 1.323, Train loss: 2.442
я вас со  ст ссоооот     оссспсттто
Epoch 9. Time: 1.372, Train loss: 2.410
я вас со  сттссооое     нспппоеооо 
Epoch 10. Time: 1.335, Train loss: 2.380
я вас со тсттсоеоо     сспппотоооло
Epoch 11. Time: 1.582, Train loss: 2.353
я вас со тсттсоеоо     сспппотоооло
Ep